# RNN (Recurrent neural networks)

A RNN can be visualized as a neural network with layers arranged vertically. Each layer represents a moment in time. An arrow connects each layer to the layer that follows, symbolizing the passage of information from one time step to the other. Another arrow enters and exits each layer, indicating the inputs and outputs at each moment.

Let’s also say you want to predict the direction that the ball was moving. So with only the information that you see on the screen, how would you do this? If you record many snapshots of the ball’s position in succession, you will have enough information to make a better prediction.


![ball](./asset/ball.gif "segment")

So to be short RNN can help to add memory into a sequence, like a ball know how the ball is moving

## In Text

In text is the same. A text is just a sequence of words

So RNN will help to learn more relationship between words

## Memory

Recurrent Neural Networks (RNNs) excel in processing sequence data and making predictions thanks to their trait I call 'sequential memory'. To gain a visceral understanding of what sequential memory entails, consider this analogy:

Visualize yourself reciting the alphabet:


A B C D E F G H I J K L M N O P Q R S T U V W X Y Z


Quite facile, wasn't it? It comes naturally because this specific sequence is ingrained in your memory from a young age.

Now, attempt to recite the alphabet backward:


Z Y X W V U T S R Q P O N M L K J I H G F E D C B A


This task is decidedly more challenging, unless you've undergone explicit practice. The sequence is unfamiliar, hence it's more difficult to recite.
Let's try another exercise. Begin at the letter 'F':


F...


Although the initial few letters might be a struggle, as soon as your brain identifies the pattern, the rest will follow effortlessly.

The rationale behind this challenge lies very much in the learning process. You're taught the alphabet as a sequence. Your brain uses sequential memory as a tool for recognizing sequence patterns more efficiently, making familiar sequences easier to recall.

The same principle applies to RNNs. Their sequential memory enables them to understand and predict patterns within sequential data.

## Recurrent Neural Networks

RNNs encapsulate the abstract concept of sequential memory in their operation. But how exactly does an RNN manifest this idea? To understand this, we need to examine a conventional neural network, also known as a feed-forward neural network. Such a network is composed of an input layer, a hidden layer, and an output layer.

![fdd](./asset/Feed_forward_neural_net.gif "segment")

How do we get a feed-forward neural network to be able to use previous information to effect later ones? What if we add a loop in the neural network that can pass prior information forward?


<img src="./asset/Feed_forward_rnn.gif" alt="image" width="auto" height="500">


Here an other representation of the full iteration in sequence

<img src="./asset/rnn_struct.png" alt="image" width="auto" height="300">

### Example

Lets take the sentence: "what time is it?"

**1**

Split into sequence (tokens)

![01](./asset/rnn01.gif "segment")

**2**

Feed “What” into the RNN. The RNN encodes “What” and produces an output.

![02](./asset/rnn02.gif "segment")

**3**

For the next step, we feed the word “time” and the hidden state from the previous step. The RNN now has information on both the word “What” and “time.”

![03](./asset/rnn03.gif "segment")

**4**

We repeat this process, until the final step. You can see by the final step the RNN has encoded information from all the words in previous steps.

![04](./asset/rnn04.gif "segment")

**5**

Since the final output was created from the rest of the sequence, we should be able to take the final output and pass it to the feed-forward layer to classify an intent.

![05](./asset/rnn05.gif "segment")

### Pseudo code

```python
rnn = RNN()
ff = FeedForwardNN()
hidden_state = [0.0, 0.0, 0.0, 0.0]
for word in sentence:
    output, hidden_state = rnn(word, hidden_state)
preduction = ff(output)
```

Here the operation do in the Linear layer

![rnn_illustrated](./asset/rnn_illustrated.gif "segment")

### Vanishing Gradient

The "vanishing gradient" problem in RNNs refers to the situation where the gradients of the loss function become so small during backpropagation that the weights of the network are barely updated, especially for the earlier layers. This happens because the gradient values, which are less than 1, are multiplied together multiple times during backpropagation through time, causing them to diminish exponentially. As a result, the network has difficulty learning and capturing long-range dependencies in the data.

![rnn gradiant](./asset/rnn_gradiant.png)

## Summary

**Advantages:**

1. **Sequential Processing:** RNNs are inherently designed for sequential data processing, making them perfect for time series prediction, natural language processing, and speech recognition.
2. **low cost inference:** RNNs tend to require fewer computational resources than Transformer models as they process input sequences step by step rather than in parallel.

**Disadvantages:**

1. **Vanishing and Exploding Gradient Problem:** During back-propagation in deep RNNs, gradients are multiplied by the weight matrix at every timestep. This can result in gradients that either explode or vanish, making it challenging to train deep RNNs.
2. **Long-term Dependencies:** RNNs struggle to learn long-term dependencies due to the vanishing gradient problem.
3. **Cannot Process in Parallel:** The sequential nature of RNNs means they cannot take advantage of modern GPUs which excel in performing parallel operations.

# RNN in python

We will be building and training a basic character-level Recurrent Neural Network (RNN) to classify words.

Specifically, we’ll train on a few thousand surnames from 18 languages of origin, and predict which language a name is from based on the spelling:

## Preparing the Data

In [ ]:
import glob
import os
import string
import unicodedata
from io import open


def findFiles(path):
    return glob.glob(path)


all_letters = string.ascii_letters + " .,;'"
n_letters = len(all_letters)


# Turn a Unicode string to plain ASCII, thanks to https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.noxrmalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )


# Read a file and split into lines
def readLines(filename):
    """
    Reads the contents of a file and returns them as a list of lines.

    :param filename: The name of the file to be read.
    :return: A list of lines read from the file.
    """
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicodeToAscii(line) for line in lines]


# Build the category_lines dictionary, a list of names per language
category_lines = {}
all_categories = []

for filename in findFiles('../dataset/names/names/*.txt'):
    category = os.path.splitext(os.path.basename(filename))[0]
    all_categories.append(category)
    lines = readLines(filename)
    category_lines[category] = lines

n_categories = len(all_categories)

print("findFiles", findFiles('../dataset/names/names/*.txt'))
print("unicodeToAscii", unicodeToAscii('Ślusàrski'))

Now we have **category_lines**, a dictionary mapping each category (language) to a list of lines (names). We also kept track of **all_categories** (just a list of languages) and **n_categories** for later reference.

In [ ]:
print(category_lines['Italian'][:5])
print("Number of categories:", n_categories)
print("Number of letters:", n_letters)

## Turning Names into tokens

Has we see in previous tutorials we will need to convert letters into tensor (vectors) with embedding layer.

So we need to convert letter into ids

In [ ]:
letter_to_id = {l: i for i, l in enumerate(all_letters)}
id_to_letter = {i: l for i, l in enumerate(all_letters)}
print(letter_to_id)

In [ ]:
def tokenizer(text: str):
    """
    Tokenizes the input text.

    :param text: The input text to be tokenized.
    :return: A list of token IDs corresponding to each letter in the input text.
    """
    return [letter_to_id[letter] for letter in text]


tokenizer('Bob')

## Creating the Network

To keep this examples simple we want use batch `[batch size, seq len]`.

But pass throw the model only one sequence (one name)


In [ ]:
import torch.nn as nn
import torch


class RNN(nn.Module):
    def __init__(self, vocab_size, hidden_size, output_size):
        """
        :param vocab_size: Size of the vocabulary
        :param hidden_size: Size of the hidden layers and embedding layer
        :param output_size: number of classes in the dataset
        """
        super(RNN, self).__init__()

        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(vocab_size, hidden_size)

        # output embedding size + hidden_size
        self.i2h = nn.Linear(hidden_size + hidden_size, hidden_size)
        self.h2o = nn.Linear(hidden_size, output_size)

    def forward(self, input_ids, hidden=None):
        """
        :param input_ids: Array of toke ids [seq_len]
        :param hidden: (optional) hidden state of previous layer
        :return: tuple predicted output [output_size] and hidden state [hidden_size]
        """
        # [seq_len, emb dim]
        embedding = self.embedding(input_ids)
        # [hidden_size]
        hidden = self.initHidden() if hidden is None else hidden
        for i in range(embedding.shape[0]):
            # Add hidden into the embedding last dimension
            # [emb dim + emb dim]
            combined = torch.cat((embedding[i], hidden), 0)
            hidden = self.i2h(combined) 
            # The tanh function is a popular choice because it maps its inputs to outputs in the range between
            # -1 and 1, maintaining a zero center, and so it helps in reducing the leaning towards extreme predictions.
            # This property helps in controlling the exploding gradients problem in the context of RNNs.
            hidden = torch.tanh(hidden)
            output = self.h2o(hidden)
        # Expand 1 dim for the loss function
        return output.unsqueeze(0), hidden

    def initHidden(self):
        return torch.zeros(self.hidden_size, requires_grad=False)

In [ ]:
n_hidden = 128
rnn = RNN(n_letters, n_hidden, n_categories)

# 1-Tokenize the text
tokens = torch.tensor(tokenizer('Bob'))
print("tokens", tokens.shape)

# 2- Creat the first hidden state
hidden = torch.zeros(n_hidden)

# 3- Pass throw the rnn network
output, next_hidden = rnn(tokens, hidden)
print("next_hidden", next_hidden.shape)
print("output", output.shape)

In [ ]:
def categoryFromOutput(output):
    """
    Get category from output softmax
    :param output: get from softmax
    :return: 
    """
    top_n, top_i = output.topk(1)
    category_i = top_i[0].item()
    return all_categories[category_i], category_i


# Get the category from the output
print(categoryFromOutput(output))

We will also want a quick way to get a training example (a name and its language):

In [ ]:
import random


def randomChoice(l):
    """
    :param l: list of elements to choose from
    :return: randomly selected element from the given list
    """
    return l[random.randint(0, len(l) - 1)]


def randomTrainingExample():
    """
    Generates a random training example.

    :return: a tuple containing the selected category, the selected line, the category tensor, and the tokenized line tensor (input_ids)
    :rtype: tuple
    """
    category = randomChoice(all_categories)
    line = randomChoice(category_lines[category])
    category_tensor = torch.tensor([all_categories.index(category)], dtype=torch.long)
    return category, line, category_tensor, torch.tensor(tokenizer(line))


for i in range(10):
    category, line, category_tensor, input_ids = randomTrainingExample()
    print('category =', category, '/ line =', line, '/ input_ids =', input_ids)

## Train model

Each loop of training will:
- Create input tokens
- Create a zeroed initial hidden state
- Read each letter in and
    - Keep hidden state for next letter
- Compare final output to target
- Back-propagate
- Return the output and loss

In [ ]:
import time
import math

n_iters = 105000
print_every = 5000

# Keep track of losses for plotting
current_loss = 0.0
all_losses = []

# If you set this too high, it might explode. If too low, it might not learn
learning_rate = 0.01
# Now all it takes to train this network is show it a bunch of examples, have it make guesses, and tell it if it’s wrong.
# For the loss function **nn.NLLLoss** is appropriate, since the last layer of the RNN is nn.LogSoftmax.
criterion = nn.CrossEntropyLoss()

n_hidden = 128
rnn = RNN(n_letters, n_hidden, n_categories)

optimizer = torch.optim.SGD(rnn.parameters(), lr=learning_rate, momentum=0.9)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
rnn.to(device)


def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


start = time.time()

for iter in range(1, n_iters + 1):
    category, line, category_tensor, input_ids = randomTrainingExample()
    # Send inputs to GPU
    input_ids = input_ids.to(device)
    category_tensor = category_tensor.to(device)
    
    hidden = rnn.initHidden().to(device)
    # Zero your gradients for every batch!
    # optimizer.zero_grad()
    rnn.zero_grad()
    # Make predictions for this batch
    output, hidden = rnn(input_ids, hidden)

    # Compute the loss and its gradients
    loss = criterion(output, category_tensor)
    loss.backward()

    # Update gradiant manualy
    # Add parameters' gradients to their values, multiplied by learning rate
    # for p in rnn.parameters():
    #     p.data.add_(p.grad.data, alpha=-learning_rate)
    
    
    # Clip the gradients
    # The torch.nn.utils.clip_grad_norm_(parameters, max_norm) function is used to scale the gradient clipping
    # prevents the "exploding gradients" problem, which can cause numerical overflow during gradient descent
    # backpropagation.
    torch.nn.utils.clip_grad_norm_(rnn.parameters(), max_norm=1.0)
    optimizer.step()
    
    
    current_loss += loss.item()

    # Print ``iter`` number, loss, name and guess
    if iter % print_every == 0:
        all_losses.append(current_loss / print_every)
        print(f'  iter {iter} loss: {current_loss / print_every:.3f}')
        current_loss = 0.0

Plot results

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.plot(all_losses)

In [ ]:
import matplotlib.ticker as ticker

# Keep track of correct guesses in a confusion matrix
confusion = torch.zeros(n_categories, n_categories)
n_confusion = 10000
softmax = nn.LogSoftmax(dim=1)

# Go through a bunch of examples and record which are correctly guessed
for i in range(n_confusion):
    
    # Get random train examples
    category, line, category_tensor, input_ids = randomTrainingExample()
    # Init the first hidden satet
    hidden = rnn.initHidden().to(device)
    input_ids = input_ids.to(device)
    output, hidden = rnn(input_ids, hidden)
    # Get the category from the output model
    guess, guess_i = categoryFromOutput(softmax(output))
    category_i = all_categories.index(category)
    confusion[category_i][guess_i] += 1

# Normalize by dividing every row by its sum
for i in range(n_categories):
    confusion[i] = confusion[i] / confusion[i].sum()

# Set up plot
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(confusion.numpy())
fig.colorbar(cax)

# Set up axes
ax.set_xticklabels([''] + all_categories, rotation=90)
ax.set_yticklabels([''] + all_categories)

# Force label at every tick
ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

# sphinx_gallery_thumbnail_number = 2
plt.show()

# LSTM in 5min

[ressources](https://towardsdatascience.com/illustrated-guide-to-lstms-and-gru-s-a-step-by-step-explanation-44e9eb85bf21)

Long Short-Term Memory (LSTM) is a type of recurrent neural network (RNN) architecture that excels in remembering long sequences of data, making it excellent for time series prediction, natural language processing, and other sequential tasks.

A recurrent neural network works by retaining a form of memory as it processes sequences. It does this by implementing a loop within the network where information can be passed from one step in the sequence to the next.

LSTMs improve on the basic RNN structure through a more complex recurrent unit which helps to control the flow of information.

LSTMs can overcome the major challenge of remembering long sequences and eliminating the long-term dependency problem which traditional RNNs face. That's why they are often used in deep learning to solve complex sequential problems.

![lstm](./asset/lstm.png)

## Forget gate

**decides what is relevant to keep**

First, we have the forget gate. This gate decides what information should be thrown away or kept. Information from the previous hidden state and information from the current input is passed through the sigmoid function. Values come out between 0 and 1. The closer to 0 means to forget, and the closer to 1 means to keep.

![forget](./asset/lstm_forget.gif "segment")

## Input Gate

**decides what information is relevant to add from the current step**

To update the cell state, we have the input gate. First, we pass the previous hidden state and current input into a sigmoid function. That decides which values will be updated by transforming the values to be between 0 and 1. 0 means not important, and 1 means important. You also pass the hidden state and current input into the tanh function to squish values between -1 and 1 to help regulate the network. Then you multiply the tanh output with the sigmoid output. The sigmoid output will decide which information is important to keep from the tanh output.

![input](./asset/lstm_input.gif "segment")

## Cell State

**Add the input with the forget**

Now we should have enough information to calculate the cell state. First, the cell state gets pointwise multiplied by the forget vector. This has a possibility of dropping values in the cell state if it gets multiplied by values near 0. Then we take the output from the input gate and do a pointwise addition which updates the cell state to new values that the neural network finds relevant. That gives us our new cell state.

![cell](./asset/lstm_cell.gif "segment")

## Output Gate

**determines what the next hidden state should be**

Last we have the output gate. The output gate decides what the next hidden state should be. Remember that the hidden state contains information on previous inputs. The hidden state is also used for predictions. First, we pass the previous hidden state and the current input into a sigmoid function. Then we pass the newly modified cell state to the tanh function. We multiply the tanh output with the sigmoid output to decide what information the hidden state should carry. The output is the hidden state. The new cell state and the new hidden is then carried over to the next time step.

![ouput](./asset/lstm_output.gif "segment")


# Code

Here a pseudocode to illustrate LSTM

![lstm_code](./asset/lstm_pseudo_code.png)


1. First, the previous hidden state and the current input get concatenated. We’ll call it combine.
2. Combine get’s fed into the forget layer. This layer removes non-relevant data.
4. A candidate layer is created using combine. The candidate holds possible values to add to the cell state.
3. Combine also get’s fed into the input layer. This layer decides what data from the candidate should be added to the new cell state.  
5. After computing the forget layer, candidate layer, and the input layer, the cell state is calculated using those vectors and the previous cell state.
6. The output is then computed.
7. Pointwise multiplying the output and the new cell state gives us the new hidden state.
